In [0]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import output
from IPython import display
import random
import copy

In [44]:
from tqdm import trange
from time import sleep
for _ in trange(10):
  sleep(0.1)

100%|██████████| 10/10 [00:01<00:00,  9.87it/s]


# Aprendizaje por refuerzo: Jugando Triqui

---



## Programando las reglas de juego

Para que un computador pueda entender un juego, primero debemos codificarlo usando números. Antes de empezar, recordemos un poco ¿Cuáles son las reglas de juego? 

1. ¿Cuántos jugadores hay? <font color='blue'>*2, uno __X__ y otro __O__* </font>   
2. ¿Cuántas casillas hay? <font color='blue'>*9* </font> 
3. ¿Cuántos posibles estados tiene cada casilla? <font color='blue'>* 3: en blanco, __X__, o __O__*</font>
4. ¿Cuántas configuraciones de tablero tenemos? <font color='blue'>* (3 estados) ^ (9 casillas) =19683 configuraciones*</font>
5. ¿Cómo podemos representarlas? <font color='blue'>* Con un vector, en el que cada entrada puede ser 0 = en blanco, 1 =  __X__ , 2 = __O__, y decidiremos que el computador siempre jugará con __X__*</font>
5. Dada una configuración, ¿Cuántas acciones podemos tomar? <font color='blue'>* Podemos poner una figura en cualquiera de las casillas libres. A lo más tenemos 9 acciones*</font> 

![Triqui](https://i.imgur.com/xZ7VyQ4.png)


Tambien registraremos nuestro estado en un vector con 9 entradas, cada una con  valores 0, 1 o 2 que corresponderan a las configuraciones 0 = en blanco, 1 = __X__ , 2 = __O__, y así podemos usar este vector para acceder a la tabla ```estado_accion```, es decir ```estado_accion[estado]``` nos dará un vector con 9 posiciones en la que cada valor nos dirá cuanto esperamos obtener si ponemos nuestra __O__  en esa posición



In [30]:
# Creamos 9 posiciones en 0, y nos aseguramos sean numeros enteros
estado = [0]*9
print(np.size(estado))

9


Dado que no todas las posiciones son factibles, las acciones posibles dependen de la configuración actual y saber si hay un ganador depende del juego, debemos crear funciones que miren a una configuración y nos digan si hay un ganador o no

In [0]:
def acciones_posibles(estado):
  """
  Esta funcion decide que casillas estan libres para poner O 
    @param[in] estado   Vector con entradas de 0, 1 o 2 que define la 
                        configuración actual
    @return    acciones Subconjunto de indices de (0,1,2,3,4,5,6,7,8) en los que
                        se puede poner una nueva O, es decir en la configuracion
                        actual estan en blanco
  """
  acciones = np.where( np.array(estado) == 0 )
  return acciones[0]

Para corroborar que nuestra función retorna lo que necesitamos podemos hacer la prueba con diferentes estados

In [117]:
#Este es la configuracion inicial, debemos poder ponerlos en todos
print(acciones_posibles(estado)) 
#Probar otras configuraciones
print(acciones_posibles([0,2,1,0,0,1,0,0,0]))

[0 1 2 3 4 5 6 7 8]
[0 3 4 6 7 8]


Para saber quien es el ganador, debemos definir las condiciones en las que una configuración hay un ganador. Sabemos si todas las entradas de 

*   una fila 
*   una columna
*   una diagonal

están llenas del mismo símbolo, entonces ese es el ganador. Pero si consideramos todas las posibles configuraciones ¿Qué problemas podemos tener?

<font color='blue'>* Pueden haber configuraciones con más de un ganador, por ejemplo una fila de __X__ y otra fila de __O__*</font>

¿Cómo los podemos solucionar? ¿Qué tan importantes son estos problemas? 
<font color='blue'>*Una opción es restringirnos a las configuraciones que solo son posibles, pero eso significa que debemos ver las 19683 opciones a mano y hacer una tabla que diga cuales son las válidas. Esto es muy extenuante. Otra forma simplemente es definir un ganador incluso para las configuraciones que no son válidas, y asegurar en otras funciones que el juego nunca llegará allí, en nuestro caso definimos que el ganador es el primero en tener toda una fila, columna o diagonal llena (en ese orden)*</font>

In [0]:
def quien_es_ganador(estado):
  """
  Esta función decide si en una configuracion hay un ganador
    @param[in] estado   Vector con entradas de 0, 1 o 2 que define la 
                        configuración actual
    @return    ganador  Regresa 0 si no hay ganador, 1 si X es ganador y 2 si O 
                        es ganador 
  """
  #Reorganizar la configuracion como el tablero de triqui
  matriz_estado = np.reshape(np.array(estado),(3,3))
  # Esta variable dira quien es el ganador
  ganador = 0 
  # 1. Revisar filas
  for i in xrange(3):
    #Si la fila no esta vacia
    if (matriz_estado[i,0] > 0):
      
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[i,0] == matriz_estado[i,1]) and 
          (matriz_estado[i,0] == matriz_estado[i,2])):
        ganador = matriz_estado[i,0]
        return ganador
  
  # 2. Revisar Columnas
  for i in xrange(3):
    #Si la columna no esta vacia
    if (matriz_estado[0,i] > 0):
      
      #Verificar que las 3 entradas de la fila son iguales
      if ((matriz_estado[0,i] == matriz_estado[1,i]) and 
          (matriz_estado[0,i] == matriz_estado[2,i])):
        ganador = matriz_estado[0,i]
        return ganador  
      
  # 3. Revisar diagonal principal
  #Si la diagonal no esta vacia
  if (matriz_estado[0,0] > 0):
    #Verificar que las 3 entradas de la fila son iguales
    if ((matriz_estado[0,0] == matriz_estado[1,1]) and 
        (matriz_estado[0,0] == matriz_estado[2,2])):
      ganador = matriz_estado[0,0]
      return ganador 
    
        
  # 3. Revisar diagonal secundaria
  #Si la diagonal no esta vacia
  if (matriz_estado[0,2] > 0):
    #Verificar que las 3 entradas de la fila son iguales
    if ((matriz_estado[0,2] == matriz_estado[1,1]) and 
        (matriz_estado[0,2] == matriz_estado[2,0])):
      ganador = matriz_estado[0,2]
      return ganador 
  
  #Si llegamos a este punto es que no hay ganador
  return ganador

Para corroborar que nuestra función retorna lo que necesitamos podemos hacer la prueba con diferentes estados

In [29]:
print("El ganador es %d"%(quien_es_ganador([1,2,1,0,2,1,0,2,1]))) #El Ganador es 2 
print("El ganador es %d"%(quien_es_ganador([0,0,0,1,1,1,2,2,2]))) #El Ganador es 1 
print("El ganador es %d"%(quien_es_ganador([1,0,1,0,2,0,0,2,1]))) #El Ganador es 0 (i.e. no hay ganador)

El ganador es 2
El ganador es 1
El ganador es 0


## Haciendo un Jugador Aleatorio

Ahora ya podemos crear un jugador aleatorio, es decir, uno que dada una configuracion, selecciona cualquiera de las posibles acciones con la misma probabilidad, e identifica cuando el o su contrincante han ganado

In [146]:
#Botones para las posiciones en el juego

display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script>
    '''))

#Clase para manejar el juego
class juego:
  def __init__(self):
    #Guarda la configuracion inicial 
    self.configuracion = [0]*9
    #Evita seguir jugando si ya existe un ganador
    self.fin_juego = False
 
  def refrescar_enemigo(self,numero):
    """
    Esta funcion refresca la configuracion incluyendo la posicion aceptada por
    el jugador
    @param[in] numero Posicion en 0:9 en el tablero definida por los botones en 
                      el juego
    """
    if not self.fin_juego:
      #Primero agrego tu posicion si es valida
      if self.configuracion[numero] == 0:
        
        self.configuracion[numero] = 1
                
        #Verifico si tu eres el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 1:
          self.imprimir()
          print("¡Tu ganaste!")
          self.fin_juego = True
          return 'OK'
        #Actualizo mi posicion
        self.refrescar_me()
        self.imprimir()
        
        #Verifico si yo soy el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 2:
          self.imprimir()
          print("¡Yo gane!")
          self.fin_juego = True
   
      else:
        print ("%d :Esa posicion ya esta ocupada"%(numero))
        
    return 'OK'
  
  def refrescar_me(self):
    """
    Esta funcion refresca la configuracion incluyendo la posicion del computador
    """
    #Verifico cuales son mis jugadas
    mis_acciones = acciones_posibles(self.configuracion)
    num_acciones = len(mis_acciones)
    
    #Si puedo jugar, selecciono una posicion al azar
    if num_acciones > 0:
      accion = random.choice(mis_acciones)
      self.configuracion[accion] = 2
    #Si no puedo jugar es porque el tablero esta lleno 
    else:
      print("Ya no hay un ganador")
      self.fin_juego = True
  
  def imprimir(self):
    """
    Esta funcion imprime el tablero en forma de matriz
    """
    for i in xrange(3):
      linea = " ";
      for j in xrange(3):
        if self.configuracion[i*3+j] == 0:
          linea += " "
        if self.configuracion[i*3+j] == 1:
          linea += "X"
        if self.configuracion[i*3+j] == 2:
          linea += "O"
        if j < 2:
          linea += "|"
      print(linea)
    print("-+-+-+-+-+")

#Creamos una nueva instancia de juego
mijuego = juego();  
 
#print(mijuego.configuracion)
output.register_callback('notebook.updatelist',mijuego.refrescar_enemigo)

En este codigo nos podemos dar cuenta que existe un gran trabajo detrás de hacer botones funcionales, y utilizamos una clase para manejar toda la información del juego, asi podemos llamar una función varias veces que modifica un objeto específico, en este caso la configuración del juego. 

Mira que los botones solo lo hacemos para que un usuario pueda interactuar con el juego, pero para los cálculos no los necesitamos

## Haciendo un jugador que aprende

La forma en la que el computador decidirá qué decisión tomar es por medio de una función de estado, es decir, escogerá la acción que lo lleve a maximizar su valor esperado. Esta es la función que descubriremos a medida que interactuamos con el sistema. Para esta la inicializaremos en 0 y crearemos una casilla de valor para cada una de las configuraciones posibles. Sin embargo tengamos en cuenta que muchas de estas configuraciones no son factibles en el juego (por ejemplo, tener todo el tablero lleno de __X__ o de __O__)

In [0]:
def inicializar_valor_estado_accion(valor_estado_accion):
  
  #Crear todas las configuraciones
  configuraciones = [[]]
  for i in xrange(9):
    nuevas_conf = []
    for j in xrange(3):
      for conf in configuraciones:
        nuevas_conf.append(conf + [j,])
    configuraciones = nuevas_conf
  
  #Update the values in each configuracion
  for conf in configuraciones:
    mis_acciones = acciones_posibles(tuple(conf))
    valor_acciones = valor_estado_accion[tuple(conf)]
    valor_acciones[mis_acciones] = 0
  return valor_estado_accion


La forma en que nuestro computador va a aprender es interactuando en contra de un jugador dummie, como el que hicimos en nuestro juego aleatorio. En vez de imprimir cada iteración y seleccionar una accion con botones, en cada iteración necesitamos informar al exterior cual es la configuración actual, si ya terminó el juego y quien gano

In [0]:
#Clase para manejar el juego
class juego_a_aprender:
  def __init__(self,quien_inicia = 1):
    self.configuracion = [0]*9
    
    #Si se decide quien inicia es la computadora
    if quien_inicia == 2:
      accion = random.choice(range(9))
      self.configuracion[accion] = 2
    
    #Evita seguir jugando si ya existe un ganador
    self.fin_juego = False
    #Guarda la lista de configuraciones
    self.conj_configuracion = [copy.deepcopy(self.configuracion)];
    #Guarda la lista de acciones
    self.acciones = []
  def refrescar_enemigo(self,numero):
    """
    Esta funcion refresca la configuracion incluyendo la posicion aceptada por
    el jugador
    @param[in] numero Posicion en 0:9 definida por el otro jugador
    """
    if not self.fin_juego:
      #Primero agrego tu posicion si es valida
      if self.configuracion[numero] == 0:
        
        self.configuracion[numero] = 1
        #Agrego tus acciones a la lista
        self.acciones.append(numero)
        
        #Verifico si tu eres el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 1:
          self.fin_juego = True
          return ganador
        
        #Actualizo mi posicion
        mi_estado = self.refrescar_me()
        
        #Si no puedo jugar es porque el tablero esta lleno 
        if mi_estado < 2:
          self.fin_juego = True
          if mi_estado == 0:
            return 0
        
        self.conj_configuracion.append(copy.deepcopy(self.configuracion))
        
        #Verifico si yo soy el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 2:
          self.fin_juego = True
          return ganador
          
    return 0
  
  def refrescar_me(self):
    """
    Esta funcion refresca la configuracion incluyendo la posicion del computador
    """
    #Verifico cuales son mis jugadas
    mis_acciones = acciones_posibles(self.configuracion)
    num_acciones = len(mis_acciones)
    
    #Si puedo jugar, selecciono una posicion al azar
    if num_acciones > 0:
      accion = random.choice(mis_acciones)
      self.configuracion[accion] = 2
      
    return num_acciones

#Creamos una nueva instancia de juego
mijuego = juego_a_aprender();  

In [0]:
def jugar(quien_inicia,opcion_optima):
  """
  Esta función interactua con refrescar enemigo del juego a aprender
  """
  enemigo = juego_a_aprender(quien_inicia)
  while not enemigo.fin_juego:
    #Vamos a seleccionar una opcion optima de acuerdo a la configuracion actual
    accion = opcion_optima(enemigo.configuracion)
    
    if accion == -1:
      break
      
    #Vamos a jugar con el enemigo
    ganador = enemigo.refrescar_enemigo(accion)
    
  return (ganador, enemigo.conj_configuracion, enemigo.acciones)

In [0]:
valor_estado_accion = np.ones((3,)*9+(9,))*-2
print(np.size(valor_estado_accion))


valor_estado_accion = inicializar_valor_estado_accion(valor_estado_accion)
  
num_estado_accion = np.zeros((3,)*9+(9,))

class juego_aprendiz_MC:
  def __init__(self,valor_estado_accion,num_estado_accion,num_iteraciones = 100,disc_factor = 0.8):
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.num_estado_accion = copy.deepcopy(num_estado_accion)
    self.num_iteraciones = num_iteraciones
    self.iteracion = 0
    self.ganancias = 0
    self.perdidas = 0
    self.empates = 0
    self.disc_factor = disc_factor
    
  def optimizar_funcion_objetivo(self):
    """
    Utilizar MC para mejorar la funcion objetivo
    """
    for i in xrange(self.num_iteraciones):
      self.iteracion = i
      quien_inicia = random.choice([1,2])
      [ganador,configuraciones,acciones] = jugar(quien_inicia, self.opcion_optima)
      precio = 0
    
      alpha = 100.
      if ganador == 1:
        precio = 1
        self.ganancias += (1.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      elif ganador == 2:
        precio = -1
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (1.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      else:
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (1.- self.empates)/alpha
        
      #print (quien_inicia,ganador,self.ganancias, self.perdidas, self.empates)  
      #print (ganador)
      for conf,accion,exponente in zip(configuraciones,acciones,xrange(len(acciones)-1,-1,-1)):
        indice = tuple(conf + [accion,])
        self.num_estado_accion[indice] += 1
        #alpha = self.num_estado_accion[indice]
        alpha = 100
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice] 
        self.valor_estado_accion[indice] += error/alpha
        
    
      
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #epsilon-greedy
    if len(acciones) > 0:
      
      u = random.random()
      
      if u * self.iteracion < 100.:
        
        mejor_accion = random.choice(acciones)
    
      else:
    
        mejor_valor = np.amax(val_acciones[acciones])
        index_accion = np.where(val_acciones[acciones] == mejor_valor )
        index_accion = index_accion[0]
    
        if len(index_accion)>1:
          mejor_accion = acciones[random.choice(index_accion)]
      
        else: 
          mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
    
mi_aprendiz = juego_aprendiz_MC(valor_estado_accion,num_estado_accion,100000)
mi_aprendiz.optimizar_funcion_objetivo()

In [13]:
print(mi_aprendiz.valor_estado_accion[tuple([0,]*9)])

[0.10798462 0.08382358 0.08644881 0.0774013  0.11744687 0.01811452
 0.45599427 0.009157   0.01940864]


In [14]:
print(valor_estado_accion[tuple([0,]*9)])

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
num_estado_accion = np.zeros((3,)*9+(9,))
mi_aprendiz2 = juego_aprendiz_MC(mi_aprendiz.valor_estado_accion,num_estado_accion,100000)

mi_aprendiz2.optimizar_funcion_objetivo()

In [16]:
print(mi_aprendiz2.valor_estado_accion[tuple([0,]*9)])

[0.15646191 0.07388466 0.13401711 0.13977236 0.15057158 0.11092394
 0.49403496 0.03817945 0.08583251]


In [0]:
num_estado_accion = np.zeros((3,)*9+(9,))
mi_aprendiz3 = juego_aprendiz_MC(mi_aprendiz2.valor_estado_accion,num_estado_accion,100000)

mi_aprendiz3.optimizar_funcion_objetivo()

In [331]:
print(mi_aprendiz3.valor_estado_accion[tuple([0,]*9)])

[ 0.3072     -0.14848     0.54793532  0.3904      0.17493333  0.16523636
  0.24576     0.064       0.18432   ]


In [0]:

mi_aprendiz4 = juego_aprendiz_MC(mi_aprendiz3.valor_estado_accion,num_estado_accion,100000)

mi_aprendiz4.optimizar_funcion_objetivo()

In [333]:
print(mi_aprendiz4.valor_estado_accion[tuple([0,]*9)])

[-0.03584     0.06034286  0.55652075  0.25813333  0.17005714  0.1264
  0.32689231 -0.008       0.05851429]


In [0]:
def reflejo_x_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.zeros((3,3),dtype=int)
  nueva_conf[0,:] = matriz_conf[2,:]
  nueva_conf[1,:] = matriz_conf[1,:]
  nueva_conf[2,:] = matriz_conf[0,:]
  return (list(np.reshape(nueva_conf,9)))

def reflejo_y_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.zeros((3,3),dtype=int)
  nueva_conf[:,0] = matriz_conf[:,2]
  nueva_conf[:,1] = matriz_conf[:,1]
  nueva_conf[:,2] = matriz_conf[:,0]
  return (list(np.reshape(nueva_conf,9)))

def traspuesta_conf(configuracion):
  matriz_conf = np.reshape(np.array(configuracion),(3,3))
  nueva_conf = np.transpose(matriz_conf)
  return (list(np.reshape(nueva_conf,9)))

def reflejo_x_accion(accion):
  if accion < 3:
    return accion + 6
  if accion > 5:
    return accion - 6
  return accion
  
def reflejo_y_accion(accion):
  if (accion % 3 == 0):
    return accion + 2
  if (accion % 3 == 2):
    return accion - 2
  return accion

def traspuesta_accion(accion):
  i = accion % 3
  j = accion / 3
  return i*3+j
  

In [24]:
Casa = [0,1,2,0,1,1,2,2,2]
print(reflejo_x_conf([0]*9))
print(reflejo_x_conf(Casa))
print(reflejo_y_conf(Casa))
print(reflejo_x_conf(reflejo_y_conf(Casa)))
Casa2 = traspuesta_conf(Casa)
print(Casa2)
print(reflejo_x_conf(Casa2))
print(reflejo_y_conf(Casa2))
print(reflejo_x_conf(reflejo_y_conf(Casa2)))
print(Casa)
print(traspuesta_accion(2))


[0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 2, 0, 1, 1, 0, 1, 2]
[2, 1, 0, 1, 1, 0, 2, 2, 2]
[2, 2, 2, 1, 1, 0, 2, 1, 0]
[0, 0, 2, 1, 1, 2, 2, 1, 2]
[2, 1, 2, 1, 1, 2, 0, 0, 2]
[2, 0, 0, 2, 1, 1, 2, 1, 2]
[2, 1, 2, 2, 1, 1, 2, 0, 0]
[0, 1, 2, 0, 1, 1, 2, 2, 2]
6


In [0]:
valor_estado_accion = np.ones((3,)*9+(9,))*-2
print(np.size(valor_estado_accion))


valor_estado_accion = inicializar_valor_estado_accion(valor_estado_accion)
  
num_estado_accion = np.zeros((3,)*9+(9,))

class juego_aprendiz_MC_2:
  def __init__(self,valor_estado_accion,num_estado_accion,num_iteraciones = 100,disc_factor = 0.8):
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.num_estado_accion = copy.deepcopy(num_estado_accion)
    self.num_iteraciones = num_iteraciones
    self.iteracion = 0
    self.ganancias = 0
    self.perdidas = 0
    self.empates = 0
    self.disc_factor = disc_factor
    
  def optimizar_funcion_objetivo(self):
    """
    Utilizar MC para mejorar la funcion objetivo
    """
    for i in xrange(self.num_iteraciones):
      self.iteracion = i
      quien_inicia = random.choice([1,2])
      [ganador,configuraciones,acciones] = jugar(quien_inicia, self.opcion_optima)
      precio = 0
    
      alpha = 100.
      if ganador == 1:
        precio = 1
        self.ganancias += (1.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      elif ganador == 2:
        precio = -1
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (1.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      else:
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (1.- self.empates)/alpha
        
      #print (quien_inicia,ganador,self.ganancias, self.perdidas, self.empates)  
       
      #print (ganador)
      for conf,accion,exponente in zip(configuraciones,acciones,xrange(len(acciones)-1,-1,-1)):
        alpha = 100
        
        
        indice = tuple(conf + [accion,])
        self.num_estado_accion[indice] += 1
        #alpha = self.num_estado_accion[indice]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice] 
        self.valor_estado_accion[indice] += error/alpha
        
        #Apply symmetries
        conf2 = reflejo_x_conf(conf)
        accion2 = reflejo_x_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf)
        accion2 = reflejo_y_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        #Con transpuesta
        conf3 = traspuesta_conf(conf)
        accion3 = traspuesta_accion(accion)
        indice2 = tuple(conf3 + [accion3,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        
        conf2 = reflejo_x_conf(conf3)
        accion2 = reflejo_x_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf3)
        accion2 = reflejo_y_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf3))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion3))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
    
      
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #epsilon-greedy
    if len(acciones) > 0:
      
      u = random.random()
      
      if u * self.iteracion < 100.:
        
        mejor_accion = random.choice(acciones)
    
      else:
    
        mejor_valor = np.amax(val_acciones[acciones])
        index_accion = np.where(val_acciones[acciones] == mejor_valor )
        index_accion = index_accion[0]
    
        if len(index_accion)>1:
          mejor_accion = acciones[random.choice(index_accion)]
      
        else: 
          mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
    
mi_aprendiz = juego_aprendiz_MC_2(valor_estado_accion,num_estado_accion,100000)
mi_aprendiz.optimizar_funcion_objetivo()

In [38]:
print(mi_aprendiz.valor_estado_accion[(0,0,0,0,0,0,0,2,0)])

[ 0.09613552  0.06064693  0.09613552  0.0153041   0.12351794  0.0153041
  0.41764943 -2.          0.41764943]


In [13]:
print(mi_aprendiz.num_estado_accion[tuple([0,]*9)])

[6.00000e+01 4.20000e+01 6.00000e+01 4.20000e+01 3.98856e+05 4.20000e+01
 6.00000e+01 4.20000e+01 6.00000e+01]


In [0]:
def conjugada_configuracion(configuracion):
  """Cambia los 1 por 2 y viceversa"""
  nueva_configuracion = []
  for i in xrange(len(configuracion)):
    if configuracion[i] == 1:
      nueva_configuracion.append(2)
    elif configuracion[i] == 2:
      nueva_configuracion.append(1)
    else:
      nueva_configuracion.append(0)
  return nueva_configuracion 

In [55]:
conjugada_configuracion(np.array([0,1,2,0,0,0,2]))

hola
[0]
[0, 2]
[0, 2, 1]
[0, 2, 1, 0]
[0, 2, 1, 0, 0]
[0, 2, 1, 0, 0, 0]
[0, 2, 1, 0, 0, 0, 1]


[0, 2, 1, 0, 0, 0, 1]

In [160]:
#Interactuar con el nuevo jugador
#Botones para las posiciones en el juego

display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))

#Clase para manejar el juego
class juego_interactuar_MC:
  def __init__(self,juego_base):
    #Guarda la configuracion inicial 
    self.configuracion = [0]*9
    self.juego_base = juego_base
    #Evita seguir jugando si ya existe un ganador
    self.fin_juego = False
    self.inicio_juego = False
 
  def refrescar_enemigo(self,numero):
    """
    Esta funcion refresca la configuracion incluyendo la posicion aceptada por
    el jugador
    @param[in] numero Posicion en 0:9 en el tablero definida por los botones en 
                      el juego
    """
    self.inicio_juego = True
    if not self.fin_juego:
      #Primero agrego tu posicion si es valida
      if self.configuracion[numero] == 0:
        
        self.configuracion[numero] = 1
                
        #Verifico si tu eres el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 1:
          self.imprimir()
          print("¡Tu ganaste!")
          self.fin_juego = True
          return 'OK'
        
        #Actualizo mi posicion
        mi_estado = self.refrescar_me()
        self.imprimir()
        
        #Verifico si yo soy el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 2:
          self.fin_juego = True
          print("¡Yo gane!")
          return 'OK'
        
        #Si no puedo jugar es porque el tablero esta lleno 
        if mi_estado < 2:
          self.fin_juego = True
          print("Ya no hay un ganador")
          return 'OK'
                
      else:
        print ("%d :Esa posicion ya esta ocupada"%(numero))
        
    return 'OK'
  
  def refrescar_me(self):
    """
    Esta funcion refresca la configuracion incluyendo la posicion del computador
    """
    #Verifico cuales son mis jugadas
    mis_acciones = acciones_posibles(self.configuracion)
    num_acciones = len(mis_acciones)
    
    #Si puedo jugar, selecciono una posicion al azar
    if num_acciones > 0:
      conj_conf = conjugada_configuracion(self.configuracion)
      accion = self.opcion_optima(conj_conf)
      self.configuracion[accion] = 2

    return num_acciones
  
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.juego_base.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    #print(val_acciones)
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #greedy
    if len(acciones) > 0:
      mejor_valor = np.amax(val_acciones[acciones])
      index_accion = np.where(val_acciones[acciones] == mejor_valor )
      index_accion = index_accion[0]
    
      if len(index_accion)>1:
        mejor_accion = acciones[random.choice(index_accion)]
      
      else: 
        mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
  
  def comenzar_juego(self):
    if not self.inicio_juego:
      print("Yo comienzo :)")
      self.refrescar_me()
      self.imprimir()
      
  def imprimir(self):
    """
    Esta funcion imprime el tablero en forma de matriz
    """
    for i in xrange(3):
      linea = " ";
      for j in xrange(3):
        if self.configuracion[i*3+j] == 0:
          linea += " "
        if self.configuracion[i*3+j] == 1:
          linea += "X"
        if self.configuracion[i*3+j] == 2:
          linea += "O"
        if j < 2:
          linea += "|"
      print(linea)
    print("-+-+-+-+-+")

#Creamos una nueva instancia de juego
mijuego2 = juego_interactuar_MC(mi_aprendiz)
 
output.register_callback('notebook.updatelist',mijuego2.refrescar_enemigo)
output.register_callback('notebook.empezar',mijuego2.comenzar_juego)

Hasta el momento puede bloquear jugadas, pero no reacciona cuando yo inicio jugando y estoy a punto de hacer una jugada. que pasa si lo enfrentamos contra si mismo?

Vamos a hacer la clase juego_a_aprender mas general

In [0]:
#Clase para manejar el juego
class juego_a_aprender_2:
  def __init__(self, juego_base,quien_inicia = 1):
    self.configuracion = [0]*9
    self.juego_base = juego_base
    
    #Si se decide quien inicia es la computadora
    if quien_inicia == 2:
      self.refrescar_me()
    
    #Evita seguir jugando si ya existe un ganador
    self.fin_juego = False
    #Guarda la lista de configuraciones
    self.conj_configuracion = [copy.deepcopy(self.configuracion)];
    #Guarda la lista de acciones
    self.acciones = []
  def refrescar_enemigo(self,numero):
    """
    Esta funcion refresca la configuracion incluyendo la posicion aceptada por
    el jugador
    @param[in] numero Posicion en 0:9 definida por el otro jugador
    """
    if not self.fin_juego:
      #Primero agrego tu posicion si es valida
      if self.configuracion[numero] == 0:
        
        self.configuracion[numero] = 1
        #Agrego tus acciones a la lista
        self.acciones.append(numero)
        
        #Verifico si tu eres el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 1:
          self.fin_juego = True
          return ganador
        
        #Actualizo mi posicion
        mi_estado = self.refrescar_me()
        
        #Si no puedo jugar es porque el tablero esta lleno 
        if mi_estado < 2:
          self.fin_juego = True
          if mi_estado == 0:
            return 0
        
        self.conj_configuracion.append(copy.deepcopy(self.configuracion))
        
        #Verifico si yo soy el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 2:
          self.fin_juego = True
          return ganador
       
    return 0
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.juego_base.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #greedy
    if len(acciones) > 0:
      mejor_valor = np.amax(val_acciones[acciones])
      index_accion = np.where(val_acciones[acciones] == mejor_valor )
      index_accion = index_accion[0]
    
      if len(index_accion)>1:
        mejor_accion = acciones[random.choice(index_accion)]
      
      else: 
        mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
  def refrescar_me(self):
    """
    Esta funcion refresca la configuracion incluyendo la posicion del computador
    """
    #Verifico cuales son mis jugadas
    mis_acciones = acciones_posibles(self.configuracion)
    num_acciones = len(mis_acciones)
    
    #Si puedo jugar, selecciono una posicion al azar
    if num_acciones > 0:
      conj_conf = conjugada_configuracion(self.configuracion)
      accion = self.opcion_optima(conj_conf)
      self.configuracion[accion] = 2
      
    return num_acciones

#Creamos una nueva instancia de juego
mijuego_profesor = juego_a_aprender_2(mi_aprendiz);  

In [0]:
def jugar_2(juego_base,quien_inicia,opcion_optima):
  """
  Esta función interactua con refrescar enemigo del juego a aprender
  """
  enemigo = juego_a_aprender_2(juego_base,quien_inicia)
  while not enemigo.fin_juego:
    #Vamos a seleccionar una opcion optima de acuerdo a la configuracion actual
    accion = opcion_optima(enemigo.configuracion)
    
    if accion == -1:
      break
      
    #Vamos a jugar con el enemigo
    ganador = enemigo.refrescar_enemigo(accion)
    
  return (ganador, enemigo.conj_configuracion, enemigo.acciones)

In [105]:
valor_estado_accion = np.ones((3,)*9+(9,))*-2
print(np.size(valor_estado_accion))


valor_estado_accion = inicializar_valor_estado_accion(valor_estado_accion)
  
num_estado_accion = np.zeros((3,)*9+(9,))

class juego_aprendiz_MC_3:
  def __init__(self,juego_base,valor_estado_accion,num_estado_accion,num_iteraciones = 100,disc_factor = 0.8):
    self.juego_base = juego_base
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.num_estado_accion = copy.deepcopy(num_estado_accion)
    self.num_iteraciones = num_iteraciones
    self.iteracion = 0
    self.ganancias = 0
    self.perdidas = 0
    self.empates = 0
    self.disc_factor = disc_factor
    
  def optimizar_funcion_objetivo(self):
    """
    Utilizar MC para mejorar la funcion objetivo
    """
    for i in xrange(self.num_iteraciones):
      self.iteracion = i
      quien_inicia = random.choice([1,2])
      [ganador,configuraciones,acciones] = jugar_2(self.juego_base,quien_inicia, self.opcion_optima)
      precio = 0
    
      alpha = 100.
      if ganador == 1:
        precio = 1
        self.ganancias += (1.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      elif ganador == 2:
        precio = -1
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (1.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      else:
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (1.- self.empates)/alpha
        
      #print (quien_inicia,ganador,self.ganancias, self.perdidas, self.empates)  
       
      #print (ganador)
      for conf,accion,exponente in zip(configuraciones,acciones,xrange(len(acciones)-1,-1,-1)):
        alpha = 100
        
        
        indice = tuple(conf + [accion,])
        self.num_estado_accion[indice] += 1
        #alpha = self.num_estado_accion[indice]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice] 
        self.valor_estado_accion[indice] += error/alpha
        
        #Apply symmetries
        conf2 = reflejo_x_conf(conf)
        accion2 = reflejo_x_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf)
        accion2 = reflejo_y_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        #Con transpuesta
        conf3 = traspuesta_conf(conf)
        accion3 = traspuesta_accion(accion)
        indice2 = tuple(conf3 + [accion3,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        
        conf2 = reflejo_x_conf(conf3)
        accion2 = reflejo_x_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf3)
        accion2 = reflejo_y_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf3))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion3))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
    
      
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #epsilon-greedy
    if len(acciones) > 0:
      
      u = random.random()
      
      if u * self.iteracion < 100.:
        
        mejor_accion = random.choice(acciones)
    
      else:
    
        mejor_valor = np.amax(val_acciones[acciones])
        index_accion = np.where(val_acciones[acciones] == mejor_valor )
        index_accion = index_accion[0]
    
        if len(index_accion)>1:
          mejor_accion = acciones[random.choice(index_accion)]
      
        else: 
          mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
    
mi_aprendiz2 = juego_aprendiz_MC_3(mi_aprendiz,valor_estado_accion,num_estado_accion,100000)
mi_aprendiz2.optimizar_funcion_objetivo()

177147


In [111]:
print(mi_aprendiz2.valor_estado_accion[(0,0,2,0,0,0,0,0,0)])

[-0.1052203  -0.10740668 -2.          0.63999993 -0.11614075 -0.10740668
 -0.08733162  0.63999993 -0.1052203 ]


In [128]:
#Interactuar con el nuevo jugador
#Botones para las posiciones en el juego

display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))


#Creamos una nueva instancia de juego
mijuego3 = juego_interactuar_MC(mi_aprendiz2)
 
output.register_callback('notebook.updatelist',mijuego3.refrescar_enemigo)
output.register_callback('notebook.empezar',mijuego3.comenzar_juego)

[-2.         -0.10740668 -0.1052203  -0.10740668 -0.11614075  0.63999993
 -0.1052203   0.63999993 -0.08733162]
 X| | 
  | |O
  | | 
-+-+-+-+-+
[-2.  0.  0. -2.  0. -2.  0.  0.  0.]
 X| | 
 X| |O
  |O| 
-+-+-+-+-+


El juego sigue sin reconocer cuando casi el opuesto va a ganar, por lo tanto vamos a entrenarlo con el mismo jugador aleatorio, pero forzando a que cuando este pueda ganar en una jugada, lo haga.

In [0]:
#Clase para manejar el juego
class juego_a_aprender_3:
  def __init__(self, quien_inicia = 1):
    self.configuracion = [0]*9
    
    #Si se decide quien inicia es la computadora
    if quien_inicia == 2:
      self.refrescar_me()
    
    #Evita seguir jugando si ya existe un ganador
    self.fin_juego = False
    #Guarda la lista de configuraciones
    self.conj_configuracion = [copy.deepcopy(self.configuracion)];
    #Guarda la lista de acciones
    self.acciones = []
  def refrescar_enemigo(self,numero):
    """
    Esta funcion refresca la configuracion incluyendo la posicion aceptada por
    el jugador
    @param[in] numero Posicion en 0:9 definida por el otro jugador
    """
    if not self.fin_juego:
      #Primero agrego tu posicion si es valida
      if self.configuracion[numero] == 0:
        
        self.configuracion[numero] = 1
        #Agrego tus acciones a la lista
        self.acciones.append(numero)
        
        #Verifico si tu eres el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 1:
          self.fin_juego = True
          return ganador
        
        #Actualizo mi posicion
        mi_estado = self.refrescar_me()
        
        #Si no puedo jugar es porque el tablero esta lleno 
        if mi_estado < 2:
          self.fin_juego = True
          if mi_estado == 0:
            return 0
        
        self.conj_configuracion.append(copy.deepcopy(self.configuracion))
        
        #Verifico si yo soy el ganador
        ganador = quien_es_ganador(self.configuracion)
        if ganador == 2:
          self.fin_juego = True
          return ganador
       
    return 0
  def opcion_optima(self,acciones):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    pos_acciones = []
    for accion in acciones:
      pos_configuracion = copy.deepcopy(self.configuracion)
      pos_configuracion[accion] = 2
      if quien_es_ganador(pos_configuracion) == 2:
        pos_acciones.append(accion)
    
    if len(pos_acciones)>1:
      return random.choice(pos_acciones)
      
    elif len(pos_acciones)==1: 
      return pos_acciones[0]
    else:
      return random.choice(acciones)
    
  def refrescar_me(self):
    """
    Esta funcion refresca la configuracion incluyendo la posicion del computador
    """
    #Verifico cuales son mis jugadas
    mis_acciones = acciones_posibles(self.configuracion)
    num_acciones = len(mis_acciones)
    
    #Si puedo jugar, selecciono una posicion al azar
    if num_acciones > 0:
      accion = self.opcion_optima(mis_acciones)
      self.configuracion[accion] = 2
      
    return num_acciones

#Creamos una nueva instancia de juego
mijuego_profesor2 = juego_a_aprender_3();  

In [0]:
def jugar_3(quien_inicia,opcion_optima):
  """
  Esta función interactua con refrescar enemigo del juego a aprender
  """
  enemigo = juego_a_aprender_3(quien_inicia)
  while not enemigo.fin_juego:
    #Vamos a seleccionar una opcion optima de acuerdo a la configuracion actual
    accion = opcion_optima(enemigo.configuracion)
    
    if accion == -1:
      break
      
    #Vamos a jugar con el enemigo
    ganador = enemigo.refrescar_enemigo(accion)
    
  return (ganador, enemigo.conj_configuracion, enemigo.acciones)

In [132]:
valor_estado_accion = np.ones((3,)*9+(9,))*-2
print(np.size(valor_estado_accion))


valor_estado_accion = inicializar_valor_estado_accion(valor_estado_accion)
  
num_estado_accion = np.zeros((3,)*9+(9,))

class juego_aprendiz_MC_4:
  def __init__(self,valor_estado_accion,num_estado_accion,num_iteraciones = 100,disc_factor = 0.8):
    self.valor_estado_accion = copy.deepcopy(valor_estado_accion)
    self.num_estado_accion = copy.deepcopy(num_estado_accion)
    self.num_iteraciones = num_iteraciones
    self.iteracion = 0
    self.ganancias = 0
    self.perdidas = 0
    self.empates = 0
    self.disc_factor = disc_factor
    
  def optimizar_funcion_objetivo(self):
    """
    Utilizar MC para mejorar la funcion objetivo
    """
    for i in xrange(self.num_iteraciones):
      self.iteracion = i
      quien_inicia = random.choice([1,2])
      [ganador,configuraciones,acciones] = jugar_3(quien_inicia, self.opcion_optima)
      precio = 0
    
      alpha = 100.
      if ganador == 1:
        precio = 1
        self.ganancias += (1.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      elif ganador == 2:
        precio = -1
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (1.- self.perdidas)/alpha
        self.empates += (0.- self.empates)/alpha
      else:
        self.ganancias += (0.- self.ganancias)/alpha
        self.perdidas += (0.- self.perdidas)/alpha
        self.empates += (1.- self.empates)/alpha
        
      #print (quien_inicia,ganador,self.ganancias, self.perdidas, self.empates)  
       
      #print (ganador)
      for conf,accion,exponente in zip(configuraciones,acciones,xrange(len(acciones)-1,-1,-1)):
        alpha = 100
        
        
        indice = tuple(conf + [accion,])
        self.num_estado_accion[indice] += 1
        #alpha = self.num_estado_accion[indice]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice] 
        self.valor_estado_accion[indice] += error/alpha
        
        #Apply symmetries
        conf2 = reflejo_x_conf(conf)
        accion2 = reflejo_x_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf)
        accion2 = reflejo_y_accion(accion)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        #Con transpuesta
        conf3 = traspuesta_conf(conf)
        accion3 = traspuesta_accion(accion)
        indice2 = tuple(conf3 + [accion3,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        
        conf2 = reflejo_x_conf(conf3)
        accion2 = reflejo_x_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_y_conf(conf3)
        accion2 = reflejo_y_accion(accion3)
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
        
        conf2 = reflejo_x_conf(reflejo_y_conf(conf3))
        accion2 = reflejo_x_accion(reflejo_y_accion(accion3))
        indice2 = tuple(conf2 + [accion2,])
        self.num_estado_accion[indice2] += 1
        #alpha = self.num_estado_accion[indice2]
        error = precio * (self.disc_factor ** exponente) - self.valor_estado_accion[indice2] 
        self.valor_estado_accion[indice2] += error/alpha
    
      
  def opcion_optima(self,configuracion):
    """Selecciona la mejor configuracion de acuerdo a la funcion de valor 
    segun estado y accion """
    #Recolectar el vector de acciones segun configuracion
    val_acciones = self.valor_estado_accion[tuple(configuracion)] 
    
    #Seleccionar las acciones posibles (una accion no es posible si la casilla 
    #esta llena)
    
    acciones = np.where( val_acciones > -2 )
    acciones = acciones[0]
    mejor_accion = 0
    #epsilon-greedy
    if len(acciones) > 0:
      
      u = random.random()
      
      if u * self.iteracion < 100.:
        
        mejor_accion = random.choice(acciones)
    
      else:
    
        mejor_valor = np.amax(val_acciones[acciones])
        index_accion = np.where(val_acciones[acciones] == mejor_valor )
        index_accion = index_accion[0]
    
        if len(index_accion)>1:
          mejor_accion = acciones[random.choice(index_accion)]
      
        else: 
          mejor_accion = acciones[index_accion[0]]
    
    return mejor_accion
    
mi_aprendiz3 = juego_aprendiz_MC_4(valor_estado_accion,num_estado_accion,100000)
mi_aprendiz3.optimizar_funcion_objetivo()

177147


In [163]:
#Interactuar con el nuevo jugador
#Botones para las posiciones en el juego

display.display(display.HTML('''
    Posiciones del juego, tu eres X, yo soy O: <br>
    <button id='button0'>0</button>
    <script>
      document.querySelector('#button0').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [0], {});
      };
    </script>
    <button id='button1'>1</button>
    <script>
      document.querySelector('#button1').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [1], {});
      };
    </script>
    <button id='button2'>2</button>
    <script>
      document.querySelector('#button2').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [2], {});
      };
    </script><br>
    <button id='button3'>3</button>
    <script>
      document.querySelector('#button3').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [3], {});
      };
    </script>
    <button id='button4'>4</button>
    <script>
      document.querySelector('#button4').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [4], {});
      };
    </script>
    <button id='button5'>5</button>
    <script>
      document.querySelector('#button5').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [5], {});
      };
    </script><br>
    <button id='button6'>6</button>
    <script>
      document.querySelector('#button6').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [6], {});
      };
    </script>
    <button id='button7'>7</button>
    <script>
      document.querySelector('#button7').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [7], {});
      };
    </script>
    <button id='button8'>8</button>
    <script>
      document.querySelector('#button8').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.updatelist', [8], {});
      };
    </script><br>
    <button id='buttonempezar'>Yo empiezo</button>
    <script>
      document.querySelector('#buttonempezar').onclick = () => {
        google.colab.kernel.invokeFunction('notebook.empezar', [], {});
      };
    </script>
    '''))


#Creamos una nueva instancia de juego
mijuego4 = juego_interactuar_MC(mi_aprendiz3)
 
output.register_callback('notebook.updatelist',mijuego4.refrescar_enemigo)
output.register_callback('notebook.empezar',mijuego4.comenzar_juego)

 O|X| 
  | | 
  | | 
-+-+-+-+-+
 O|X| 
  |X| 
  |O| 
-+-+-+-+-+
 O|X| 
 O|X| 
  |O|X
-+-+-+-+-+
 O|X| 
 O|X|X
 O|O|X
-+-+-+-+-+
¡Yo gane!
